In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

MONTH = ['01월', '02월', '03월', '04월', '05월', '06월', '07월']
total_pop = np.random.randint(40, 100, size = len(MONTH))
male_pop = [np.random.randint(1, t) for t in total_pop]
female_pop = total_pop - male_pop
print(f"total_pop: {total_pop}, type(total_pop): {type(total_pop)}")
print(f"male_pop: {total_pop}, type(male_pop): {type(male_pop)}")
print(f"female_pop: {total_pop}, type(female_pop): {type(female_pop)}")
df = pd.DataFrame({
    '월' : MONTH,
    '총인구' : total_pop,
    '남자' : male_pop,
    '여자': female_pop
})
df

total_pop: [72 58 96 80 92 80 45], type(total_pop): <class 'numpy.ndarray'>
male_pop: [72 58 96 80 92 80 45], type(male_pop): <class 'list'>
female_pop: [72 58 96 80 92 80 45], type(female_pop): <class 'numpy.ndarray'>


,월,총인구,남자,여자
0,01월,72,64,8
1,02월,58,36,22
2,03월,96,58,38
3,04월,80,71,9
4,05월,92,81,11
5,06월,80,64,16
6,07월,45,2,43


In [4]:
# 월별 슬라이더의 프레임을 구성하기 위해 각 월에 대한 트레이스 준비
# 슬라이더의 각 스텝과 프레임 이름을 매칭 하기 위한 월 라벨 목록
frames = []
for M in MONTH:
    row = df.loc[df['월'] == M].iloc[0]
    frames.append(go.Frame(
        name = M,          # 프레임 고유 이름, 슬라이더 step의 첫번째 요소와 동일한 값이어야 함
        data = [           # 프레임에서 표현할 트레이스,
            go.Bar(x = ['총인구'], y = [row['총인구']]),
            go.Bar(x = ['남자'],  y = [row['남자']]),
            go.Bar(x = ['여자'],  y = [row['여자']]),
        ]
    ))

# 초기 상태 설정(1월만 보이게)
init_row = df.iloc[0]

# 전체 트레이스 컨테이너, 초기 data, frames, layout 할당
fig = go.Figure(
    data = [
        go.Bar(x = ['총인구'], y = [init_row['총인구']], name = '총인구', visible = True, marker = {'color' : '#30C75A'},),
        go.Bar(x = ['남자'], y = [init_row['남자']], name = '남자', visible = False, marker = {'color' : '#3148C7'},),
        go.Bar(x = ['여자'], y = [init_row['여자']], name = '여자', visible = False, marker = {'color' : '#D63626'},),
    ],
    frames = frames
)

# 슬라이더(월 이동)
sliders = [{
    'active' : 0,                 # 처음 선택할 step의 인덱스
    'x' : 0.08, 'y' : -0.12,      # 하단 배치
    'len' : 0.84,                 # 슬라이더 길이
    'currentvalue' : {            # 현재 값 표시 영역, prefix/suffix : 값의 앞/뒤에 이어 붙일 문장 작성
        'prefix' : '월: '
    },
    'steps' : [{                  # 슬라이더의 각 칸을 의미하는 리스트[{step1}, {step2}, ...]
        'method' : 'animate',     # 실행할 동작
        'args' : [          # 각 step의 세부 설정
            [M],            # 재생할 프레임 목록, 이름이 M인 프레임으로 전환, [None]으로 할당시 순서대로 재생
            {
                 'mode' : 'immediate',
                 'frame' : {'duration' : 0, 'redraw' : True}, # duration : 프레임간격(밀리초, 1000 == 1초), redraw : 매번 전체 트레이스 다시
                                                                                                           #  False일 경우 변한 부분만 다시
            }
        ],
        'label' : M
    } for M in MONTH]
}]

fig.update_layout(
    title = '슬라이더 테스트',
    sliders = sliders,
    yaxis_title = '인구수',
    yaxis = {'range' : [0, 100]}
)
fig.show()

In [7]:
# 재생/일시정지 버튼
play_pause = {
    "type" : "buttons",
    "direction" : 'right',
    'x' : 0.0, 'y' : -0.02, 'xanchor' : "left", 'yanchor' : 'top',
    "buttons" : [{
        "label" : "▶ 재생 ",
        "method" : "animate",
        "args" : [None, {"frame" : {"duration" : 1000, "redraw" : True}, "fromcurrent" : True, "transition" : {"duration" : 200}}]
        },
        {
            "label" : "❚❚ 일시정지",
            "method" : "animate",
            "args" : [[None], {"frame" : {"duration" : 0, "redraw" : False}, "mode" : "immediate"}]
        }]
}

fig.update_layout(
    updatemenus = [play_pause]

)
fig.show()

In [8]:
# 전환 버튼(총/남/여) - traces의 visible 상태를 제어
# - 각 프레임은 3개의 trace(총/남/여) 데이터를 모두 가지고 있고,
# - 버튼은 어떤 trace만 보이게 할지 토글.
metric_buttons = {
    'type' : "buttons",
    'direction' : "right",
    'x' : 1.0, 'y' : 1.2, 'xanchor' : 'right', 'yanchor' : 'top',
    'showactive' : True,
    "buttons" : [
        {"label" : "총인구", "method" : "update", "args" : [{"visible" : [True, False, False], }]},
        {"label" : "남자", "method" : "update", "args" : [{"visible" : [False, True, False], }]},
        {"label" : "여자", "method" : "update", "args" : [{"visible" : [False, False, True], }]},
        {"label" : "한꺼번에 보기", "method" : "update", "args" : [{"visible" : [True, True, True], }]}
    ]
}

fig.update_layout(
    updatemenus = [ play_pause, metric_buttons,],
    margin = {'t' : 110, 'b' : 100, 'l' : 40, 'r' : 40}

)
fig.show()